# Facility location Problems

## UFLP

### Single level

In [10]:
%%file "./pkg/uflp_cplex.py"

import cplex
import numpy as np              # mathematic tools library
import networkx as nx           # network representation library

def uflp_cplex(I,J,
            W,FCT,
            relaxation=False,path=None):

    #####################################################################
    # Objective function
    
    obj1 = [W[i][j]  for i in range(I) for j in range(J)]
    obj2 = [FCT[j] for j in range(J)]
    
    ## variables name
    Ys = [ "Y_" + str(i) + "_" + str(j) for i in range(I) for j in range(J)]
    N_icps = ["N_icp_" + str(j) for j in range(J)]

    ## Objective function sum aggregation
    obj = obj1 + obj2
    colnames = Ys + N_icps
    if relaxation:
        types    = "C" * (I*J+J) #Integrality constraint
    else:
        types    = "I" * (I*J+J) #Integrality constraint

    #####################################################################
    # Constraints
    
    c1 = [[["Y_" + str(i) + "_" + str(j) for j in range(J)], [1 for j in range(J)]] for i in range(I)]
    c3 = [[["Y_" + str(i) + "_" + str(j),"N_icp_" + str(j)], [1,-1]] for i in range(I) for j in range(J)]
    
    s1 = "E" * I
    s3 = "L" * (I*J)
    
    r1 = [1 for i in range(I)]
    r3 = [0 for i in range(I) for j in range(J)]
    
    rows = c1+c3
    senses = s1+s3
    rhs =  r1+r3
    
    #####################################################################
    # Bounds
    ub = [1 for i in range(I*J+J)]
    lb = [0 for i in range(I*J+J)]

    #####################################################################
    # Creating problem

    prob = cplex.Cplex()
    ## Objective function sense
    prob.objective.set_sense(prob.objective.sense.minimize)
    ## Objective function
    prob.variables.add(obj=obj,ub=ub,lb=lb,names=colnames,types=types)
    ## Constraintes
    prob.linear_constraints.add(lin_expr=rows,senses=senses, rhs=rhs)

    #####################################################################
    # Saving the linear problem formulation into a file
    if path:
        prob.write(path) # print the formulation into a file

    #####################################################################
    # Solving problem
    prob.solve()

    #####################################################################
    # Extract solution
    solution = prob.solution.get_values()
    Y = np.reshape(solution[0:I*J],(I,J))
    N_icp = solution[I*J:I*J+J]

    return prob, Y, N_icp

Overwriting ./pkg/uflp_cplex.py


### Two level

In [8]:
%%file "./pkg/two_lvl_uflp_cplex.py"

import cplex
import numpy as np              # mathematic tools library
import networkx as nx           # network representation library

def two_lvl_uflp_cplex(I,J,C,
            W,FCT,FCR,D,
            relaxation=False,path=None):

    #####################################################################
    # Objective function
    
    obj1 = [W[i][j]  for i in range(I) for j in range(J)]
    objx = [D[j][c]  for j in range(J) for c in range(C)]
    obj2 = [FCT[j] for j in range(J)]
    obj5 = [FCR[c] for c in range(C)]
    

    ## variablesnames
    Ys = [ "Y_" + str(i) + "_" + str(j) for i in range(I) for j in range(J)]
    Xs = [ "X_" + str(j) + "_" + str(c) for j in range(J) for c in range(C)]
    N_icps = ["N_icp_" + str(j) for j in range(J)]
    N_crcs = ["N_crc_" + str(c) for c in range(C)]

    ## Objective function sum aggregation
    obj = obj1 + objx +obj2 + obj5
    colnames = Ys + Xs + N_icps + N_crcs
    if relaxation:
        types    = "C" * (I*J+J*C+J+C) #Integrality constraint
    else:
        types    = "I" * (I*J+J*C+J+C) #Integrality constraint

    #####################################################################
    # Constraints
    
    c1 = [[["Y_" + str(i) + "_" + str(j) for j in range(J)], [1 for j in range(J)]] for i in range(I)]
    c2 = [[["X_" + str(j) + "_" + str(c) for c in range(C)], [1 for c in range(C)]] for j in range(J)]
    c3 = [[["Y_" + str(i) + "_" + str(j),"N_icp_" + str(j)], [1,-1]] for i in range(I) for j in range(J)]
    c4 = [[["X_" + str(j) + "_" + str(c),"N_crc_" + str(c)], [1,-1]] for j in range(J) for c in range(C)]
    
    s1 = "E" * I
    s2 = "L" * J
    s3 = "L" * (I*J)
    s4 = "L" * (J*C)
    
    r1 = [1 for i in range(I) ]
    r2 = [1 for j in range(J) ]
    r3 = [0 for i in range(I) for j in range(J)]
    r4 = [0 for j in range(J) for c in range(C)]
    
    rows = c1+c2+c3+c4
    senses = s1+s2+s3+s4
    rhs =  r1+r2+r3+r4
    
    #####################################################################
    # Bounds
    ub = [1 for i in range(I*J+J*C+J+C)]
    lb = [0 for i in range(I*J+J*C+J+C)]

    #####################################################################
    # Creating problem

    prob = cplex.Cplex()
    ## Objective function sense
    prob.objective.set_sense(prob.objective.sense.minimize)
    ## Objective function
    prob.variables.add(obj=obj,ub=ub,lb=lb,names=colnames,types=types)
    ## Constraintes
    prob.linear_constraints.add(lin_expr=rows,senses=senses, rhs=rhs)

    #####################################################################
    # Saving the linear problem formulation into a file
    if path:
        prob.write(path) # print the formulation into a file

    #####################################################################
    # Solving problem
    prob.solve()

    #####################################################################
    # Extract solution
    solution = prob.solution.get_values()
    Y = np.reshape(solution[0:I*J],(I,J))
    X = np.reshape(solution[I*J:I*J+J*C],(J,C))
    N_icp = solution[I*J+J*C:I*J+J*C+J]
    N_crc = solution[I*J+J*C+J:I*J+J*C+J+C]

    return prob, X, Y, N_icp, N_crc

Overwriting ./pkg/two_lvl_uflp_cplex.py


## CFLP

### Single level

In [7]:
%%file "./pkg/cflp_cplex.py"

import cplex
import numpy as np              # mathematic tools library
import networkx as nx           # network representation library

def cflp_cplex(I,J,
            W,FCT,U,Q,
            relaxation=False,path=None):

    #####################################################################
    # Objective function
    
    obj1 = [W[i][j]  for i in range(I) for j in range(J)]
    obj2 = [FCT[j] for j in range(J)]
    
    ## variablesnames
    Ys = [ "Y_" + str(i) + "_" + str(j) for i in range(I) for j in range(J)]
    N_icps = ["N_icp_" + str(j) for j in range(J)]

    ## Objective function sum aggregation
    obj = obj1 + obj2
    colnames = Ys + N_icps
    if relaxation:
        types    = "C" * (I*J+J) #Integrality constraint
    else:
        types    = "I" * (I*J+J) #Integrality constraint

    #####################################################################
    # Constraints
    
    c1 = [[["Y_" + str(i) + "_" + str(j) for j in range(J)], [1 for j in range(J)]] for i in range(I)]
    #c3 = [[["Y_" + str(i) + "_" + str(j),"N_icp_" + str(j)], [1,-1]] for i in range(I) for j in range(J)]
    cq = [[["Y_" + str(i) + "_" + str(j) for i in range(I)]+["N_icp_" + str(j)],[U[i] for i in range(I)]+[-Q[j]]] for j in range(J)]
    
    s1 = "E" * I
    #s3 = "L" * (I*J)
    sq = "L" * J
    
    r1 = [1 for i in range(I)]
    #r3 = [0 for i in range(I) for j in range(J)]
    rq = [0 for j in range(J)]
    
    rows = c1+cq
    senses = s1+sq
    rhs =  r1+rq
    
    #####################################################################
    # Bounds
    ub = [1 for i in range(I*J+J)]
    lb = [0 for i in range(I*J+J)]

    #####################################################################
    # Creating problem

    prob = cplex.Cplex()
    ## Objective function sense
    prob.objective.set_sense(prob.objective.sense.minimize)
    ## Objective function
    prob.variables.add(obj=obj,ub=ub,lb=lb,names=colnames,types=types)
    ## Constraintes
    prob.linear_constraints.add(lin_expr=rows,senses=senses, rhs=rhs)

    #####################################################################
    # Saving the linear problem formulation into a file
    if path:
        prob.write(path) # print the formulation into a file

    #####################################################################
    # Solving problem
    prob.solve()

    #####################################################################
    # Extract solution
    solution = prob.solution.get_values()
    Y = np.reshape(solution[0:I*J],(I,J))
    N_icp = solution[I*J:I*J+J]

    return prob, Y, N_icp

Overwriting ./pkg/cflp_cplex.py


### Two level (TLHCFL)

In [ ]:
%%file "./pkg/two_lvl_cflp_cplex.py"

import cplex
import numpy as np              # mathematic tools library
import networkx as nx           # network representation library

def two_lvl_cflp_cplex(I,J,C,
            W,FCT,FCR,D,U,Qicp,Qcrc
            relaxation=False,path=None):

    #####################################################################
    # Objective function
    
    obj1 = [W[i][j]  for i in range(I) for j in range(J)]
    objx = [D[j][c]  for j in range(J) for c in range(C)]
    obj2 = [FCT[j] for j in range(J)]
    obj5 = [FCR[c] for c in range(C)]
    

    ## variablesnames
    Ys = [ "Y_" + str(i) + "_" + str(j) for i in range(I) for j in range(J)]
    Xs = [ "X_" + str(j) + "_" + str(c) for j in range(J) for c in range(C)]
    N_icps = ["N_icp_" + str(j) for j in range(J)]
    N_crcs = ["N_crc_" + str(c) for c in range(C)]
    V = ["V_" + str(j) for j in range(J)]

    ## Objective function sum aggregation
    obj = obj1 + objx +obj2 + obj5
    colnames = Ys + Xs + N_icps + N_crcs + V
    if relaxation:
        types    = "C" * (I*J+J*C+J+C+J) #Integrality constraint
    else:
        types    = "I" * (I*J+J*C+J+C+J) #Integrality constraint

    #####################################################################
    # Constraints
    
    c1 = [[["Y_" + str(i) + "_" + str(j) for j in range(J)], [1 for j in range(J)]] for i in range(I)]
    c2 = [[["X_" + str(j) + "_" + str(c) for c in range(C)], [1 for c in range(C)]] for j in range(J)]
    
    c3 = [[["Y_" + str(i) + "_" + str(j) for i in range(I)]+["N_icp_" + str(j)],[U[i] for i in range(I)]+[-Qicp[j]]] for j in range(J)]
    
    c4 = [[["Y_" + str(i) + "_" + str(j) for i in range(I)]+["N_icp_" + str(j)],[U[i] for i in range(I)]+[-Qicp[j]]] for j in range(J)]
    
    s1 = "E" * I
    s2 = "E" * J
    s3 = "L" * J
    s4 = "L" * C
    
    r1 = [1 for i in range(I) ]
    r2 = [1 for j in range(J) ]
    r3 = [0 for j in range(J)]
    r4 = [0 for c in range(C)]
    
    rows = c1+c2+c3+c4
    senses = s1+s2+s3+s4
    rhs =  r1+r2+r3+r4
    
    #####################################################################
    # Bounds
    ub = [1 for i in range(I*J+J*C+J+C)]
    lb = [0 for i in range(I*J+J*C+J+C)]

    #####################################################################
    # Creating problem

    prob = cplex.Cplex()
    ## Objective function sense
    prob.objective.set_sense(prob.objective.sense.minimize)
    ## Objective function
    prob.variables.add(obj=obj,ub=ub,lb=lb,names=colnames,types=types)
    ## Constraintes
    prob.linear_constraints.add(lin_expr=rows,senses=senses, rhs=rhs)

    #####################################################################
    # Saving the linear problem formulation into a file
    if path:
        prob.write(path) # print the formulation into a file

    #####################################################################
    # Solving problem
    prob.solve()

    #####################################################################
    # Extract solution
    solution = prob.solution.get_values()
    Y = np.reshape(solution[0:I*J],(I,J))
    X = np.reshape(solution[I*J:I*J+J*C],(J,C))
    N_icp = solution[I*J+J*C:I*J+J*C+J]
    N_crc = solution[I*J+J*C+J:I*J+J*C+J+C]

    return prob, X, Y, N_icp, N_crc

# Routing Problems